<a href="https://colab.research.google.com/github/aditipabal/worldindicator-europe/blob/main/FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import geopandas as gpd

# from zip
linkGitZip="https://raw.githubusercontent.com/aditipabal/worldindicator-europe/refs/heads/main/worldindicators.json" #<- github location
gdf1 = gpd.read_file(linkGitZip)


In [5]:
gdf1.info()
gdf1.head()


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   Country       172 non-null    object  
 1   iso2          171 non-null    object  
 2   iso3          172 non-null    object  
 3   region        172 non-null    object  
 4   fragility     172 non-null    float64 
 5   co2           172 non-null    int64   
 6   sq_km         172 non-null    int32   
 7   num_airports  172 non-null    int32   
 8   population    172 non-null    int32   
 9   mobiles       172 non-null    int32   
 10  geometry      172 non-null    geometry
dtypes: float64(1), geometry(1), int32(4), int64(1), object(4)
memory usage: 12.2+ KB


,Country,iso2,iso3,region,fragility,co2,sq_km,num_airports,population,mobiles,geometry
0,ANTIGUA AND BARBUDA,AG,ATG,CENTRAL AMERICA AND THE CARIBBEAN,54.4,729000,443,4,102634,184000,"MULTIPOLYGON (((-61.73889 17.54055, -61.75195 ..."
1,AFGHANISTAN,AF,AFG,SOUTH ASIA,105.0,7893000,652230,68,40121552,23000000,"MULTIPOLYGON (((61.27656 35.60725, 61.29638 35..."
2,ALGERIA,DZ,DZA,AFRICA,75.4,151633000,2381740,95,47022473,51500000,"MULTIPOLYGON (((-5.15213 30.18047, -5.13917 30..."
3,AZERBAIJAN,AZ,AZE,MIDDLE EAST,73.2,35389000,86600,32,10650239,11000000,"MULTIPOLYGON (((46.54037 38.87559, 46.49554 38..."
4,ALBANIA,AL,ALB,EUROPE,58.9,3794000,28748,3,3107100,2610000,"MULTIPOLYGON (((20.79192 40.43154, 20.78722 40..."


In [8]:
import pandas as pd
gdf1['mobiles'] = pd.to_numeric(gdf1['mobiles'], errors='coerce')


In [11]:
europe = gdf1[gdf1['Country'] == 'Europe']


In [12]:
gdf1 = gdf1[gdf1.is_valid]


In [14]:

import geopandas as gpd
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import numpy as np

# --- 4. CHOROPLETH MAP ---
choropleth = folium.Map(location=[54, 15], zoom_start=4, tiles='cartodbpositron')

folium.Choropleth(
    geo_data=europe,
    data=europe,
    columns=['Country', 'mobiles'],
    key_on='feature.properties.Country',
    fill_color='Blues',
    fill_opacity=0.8,
    line_opacity=0.2,
    legend_name='Mobile Subscribers (total)',
).add_to(choropleth)

folium.LayerControl().add_to(choropleth)
choropleth.save("choropleth_map.html")


In [15]:
# --- 5. PROPORTIONAL SYMBOLS MAP ---
prop_map = folium.Map(location=[54, 15], zoom_start=4, tiles='cartodbpositron')

for _, row in europe.iterrows():
    if row.geometry.geom_type == 'MultiPolygon':
        lon, lat = row.geometry.centroid.x, row.geometry.centroid.y
    else:
        lon, lat = row.geometry.centroid.x, row.geometry.centroid.y

    folium.CircleMarker(
        location=[lat, lon],
        radius=np.sqrt(row['mobiles']) / 1000,  # adjust scale
        color='red',
        fill=True,
        fill_opacity=0.6,
        popup=f"{row['Country']}: {int(row['mobiles']):,}"
    ).add_to(prop_map)

prop_map.save("proportional_symbols_map.html")

In [16]:
# --- 6. DOT DENSITY MAP ---
# Create random points within country polygons
dot_map = folium.Map(location=[54, 15], zoom_start=4, tiles='cartodbpositron')
marker_cluster = MarkerCluster().add_to(dot_map)

# We'll generate one dot per N mobiles to visualize density (scaled down for performance)
for _, row in europe.iterrows():
    try:
        if row.geometry.geom_type == 'MultiPolygon':
            geom = row.geometry.geoms[0]
        else:
            geom = row.geometry
        n_points = int(row['mobiles'] / 5_000_000)  # 1 point per 5M subscribers
        if n_points < 1:
            n_points = 1

        # Random points within country
        minx, miny, maxx, maxy = geom.bounds
        for _ in range(n_points):
            lon = np.random.uniform(minx, maxx)
            lat = np.random.uniform(miny, maxy)
            if geom.contains(gpd.points_from_xy([lon], [lat])[0]):
                folium.CircleMarker(
                    location=[lat, lon],
                    radius=1,
                    color='blue',
                    fill=True,
                    fill_opacity=0.8
                ).add_to(marker_cluster)
    except Exception as e:
        print(f"Skipped {row['Country']} due to: {e}")

dot_map.save("dot_density_map.html")

In [17]:
print("✅ 3 maps created and saved:")
print(" - choropleth_map.html")
print(" - proportional_symbols_map.html")
print(" - dot_density_map.html")

✅ 3 maps created and saved:
 - choropleth_map.html
 - proportional_symbols_map.html
 - dot_density_map.html
